In [1]:
import csv
!pip install wandb
import wandb
import heapq
import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
import matplotlib.ticker as ticker
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=451b2014d9fe26d2c2721e053166cb1f2b32bc4304fab6829a5472827fed2c91
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/aksharantar_dataset/tel/'

/content/drive/MyDrive/aksharantar_dataset/tel


In [16]:
tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_train.csv")
test_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_test.csv")
val_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_valid.csv")
read_tsv = csv.reader(tsv_file)
test_read_tsv = csv.reader(test_tsv_file)
val_read_tsv = csv.reader(val_tsv_file)

In [17]:
train_X, train_Y = [], []
for i in read_tsv:
    train_X.append(i[0])
    train_Y.append(i[1])
    
test_X,test_Y = [], []
for i in test_read_tsv:
    test_X.append(i[0])
    test_Y.append(i[1])
    
val_X, val_Y = [], []
for i in val_read_tsv:
    val_X.append(i[0])
    val_Y.append(i[1])
    


test_X, test_Y = np.array(test_X), np.array(test_Y)
t_range = test_Y.shape[0]
val_X, val_Y = np.array(val_X),  np.array(val_Y)
v_range = val_Y.shape[0]
train_X, train_Y = np.array(train_X), np.array(train_Y)
tr_range = train_Y.shape[0]
for i in range(t_range):
    test_Y[i] = "\t" + test_Y[i] + "\n"
    
for i in range(v_range):
    val_Y[i] = "\t" + val_Y[i] + "\n"

for i in range(tr_range):
    train_Y[i] = "\t" + train_Y[i] + "\n"
ch_end = " "

In [18]:
val_input_corpus, output_corpus, input_corpus,  val_output_corpus = set(), set(), set(), set()

for word in train_Y:
    for char in word:
        if char not in output_corpus:
            output_corpus.add(char)

for word in train_X:
    for char in word:
        if char not in input_corpus:
            input_corpus.add(char)

output_corpus.add(ch_end)
input_corpus.add(ch_end)


for word in val_Y:
    for char in word:
        if char not in val_output_corpus:
            val_output_corpus.add(char)

output_corpus,  input_corpus = sorted(list(output_corpus)), sorted(list(input_corpus))
num_encoder_tokens =  len(input_corpus)  

for word in val_X:
    for char in word:
        if char not in val_input_corpus:
            val_input_corpus.add(char)

num_decoder_tokens = len(output_corpus)


In [19]:
max_decoder_seq_length,max_encoder_seq_length = max([len(txt) for txt in train_Y]), max([len(txt) for txt in train_X]) + 2

In [20]:
print("Number of unique input tokens:", num_encoder_tokens)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("Number of samples:", len(train_X))
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 27
Max sequence length for outputs: 21
Number of samples: 51200
Max sequence length for inputs: 30
Number of unique output tokens: 65


In [21]:
batch_size, num_epochs, learning_rate = 32, 10, 0.001
load_model, training = False, False
input_size_encoder, input_size_decoder = num_encoder_tokens, num_decoder_tokens
output_size = num_decoder_tokens
encoder_embedding_size, decoder_embedding_size = 256, 256
hidden_size = 256  # Needs to be the same for both RNN's
num_enc_layers, num_dec_layers = 1, 1
enc_dropout, dec_dropout = 0.1, 0.1
d_type, td_type = "int64", torch.int64

In [22]:
train_ln = len(train_X)
val_ln = len(val_X)
input_char_index, output_char_index = dict([(char, i) for i, char in enumerate(input_corpus)]), dict([(char, i) for i, char in enumerate(output_corpus)])
input_data, target_data = np.zeros((max_encoder_seq_length,train_ln), dtype= d_type), np.zeros((max_decoder_seq_length,train_ln), dtype=d_type)
c_end = " "
input_data_val, target_data_val = np.zeros((max_encoder_seq_length,val_ln), dtype=d_type), np.zeros((max_decoder_seq_length,val_ln), dtype=d_type)
for i, (x, y) in enumerate(zip(val_X, val_Y)):
    t_n = i+ val_ln
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
    t_n = t+1       
    target_data_val[t_n :,i] = output_char_index[c_end]
    t_n = 0
    for t, char in enumerate(x):
        input_data_val[t, i] = input_char_index[char]
    t_n = t+1   
    input_data_val[t_n :,i] = input_char_index[c_end] 
t_n =0    
for i, (x, y) in enumerate(zip(train_X, train_Y)):
    t_n = i+ val_ln
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
    t_n  = t+1        
    target_data[t_n :,i] = output_char_index[c_end]
    t_n =0
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    t_n = t+1   
    input_data[t_n :,i] = input_char_index[c_end]

In [23]:
# convertin numpy arrays to tensors
input_data_val, target_data_val = torch.tensor(input_data_val,dtype = td_type), torch.tensor(target_data_val,dtype = td_type)
input_data, target_data = torch.tensor(input_data,dtype = td_type), torch.tensor(target_data,dtype = td_type)

In [24]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_target_char_index,reverse_input_char_index, t_z = dict((i, char) for char, i in output_char_index.items()), dict((i, char) for char, i in input_char_index.items()), 0

In [25]:
class Encoder(nn.Module): 
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        drop_par = dropout
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size, = num_layers, hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

    def forward(self, x): # x shape: (seq_length, N) where N is batch size
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (seq_length, N, embedding_size)
        outputs, (hidden, cell) = self.rnn(self.dropout(drop_par)) # outputs shape: (seq_length, N, hidden_size)
        return hidden, cell

class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):

        super(Seq2Seq, self).__init__()  
        self.decoder, self.encoder = decoder, encoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        batch_size, target_len, target_vocab_size = source.shape[1], target.shape[0], num_decoder_tokens

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]
        hidden, cell = self.encoder(source)

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t], best_guess = output, output.argmax(1)
            # Get the best word the Decoder predicted (index in the vocabulary)
            x = best_guess if random.random() >= teacher_force_ratio else target[t]

        return outputs

class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        drop_par, hidden_layer_size = dropout, hidden_size
        self.dropout,self.num_layers, self.hidden_size = nn.Dropout(drop_par),  num_layers, hidden_layer_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout= drop_par)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell): # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x  = x.unsqueeze(0) 
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (1, N, embedding_size)
        outputs, (hidden, cell) = self.rnn(self.dropout(drop_par), (hidden, cell)) # outputs shape: (1, N, hidden_size)
        #predictions = self.fc(outputs)
        
        
        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = self.fc(outputs).squeeze(0)

        return predictions, hidden, cell

In [26]:
def beam_search(model, word, input_char_index, output_char_index, reverse_input_char_index,reverse_target_char_index, max_encoder_seq_length, 
                max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):


    # Encode the input word
    data, word_t = np.zeros((max_encoder_seq_length, 1), dtype= d_type), ''
    t_z , bw = 0, beam_width
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1
    data[t_z:, 0] = input_char_index[c_end]

    data = torch.tensor(data, dtype= td_type ).to(device)

    with torch.no_grad():
        hidden,cell = model.encoder(data)

    # Initialize beam
    out_t = output_char_index['\t']
    out_reshape = np.array(out_t).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]  # [(score, sequence, hidden)]

    for _ in range(max_decoder_seq_length):
      candidates = []
      for score, seq, hidden in beam:
          # last_token = seq[-1].item()
          if seq[-1].item() == output_char_index['\n']:
              # If the sequence ends with the end token, add it to the candidates
              candidates.append((score, seq, hidden))
              continue
          lt_reshape = np.array(seq[-1].item()).reshape(1,) # last_token = seq[-1].item()
          hidden_upar = hidden.squeeze(0)
          x = torch.tensor(lt_reshape).to(device)
          output, hidden,cell = model.decoder(x, hidden_upar,cell)
          probabilities = F.softmax(output, dim=1)
          cal_cand = 0
          # Get the top-k probabilities and tokens
          topk_probs, topk_tokens = torch.topk(probabilities, k=bw)

          for prob, token in zip(topk_probs[0], topk_tokens[0]):
              cal_cand = prob
              n_hidden = hidden.clone()
              new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
              ln_ns = len(new_seq)
              ln_pf = ((ln_ns - 1) / 5)
              new_hidden = n_hidden.unsqueeze(0)
              #length_penalty_factor = ln_pf ** length_penalty  # Adjust penalty factor as needed
              candidate_cal = score + torch.log(prob).item() / (ln_pf ** length_penalty)
              candidates.append((candidate_cal, new_seq, new_hidden))

      beam = heapq.nlargest(bw, candidates, key=lambda x: x[0])# Select top-k candidates based on the accumulated scores

      
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0]) # Select the best sequence from the beam as the output
    
    cal_score = best_score
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[1:-1]])
    cal_score = 0
    return word_t

In [28]:
num_epochs, batch_size, learning_rate = 2, 32, 0.001
load_model, training= False, False
input_size_encoder, input_size_decoder = num_encoder_tokens, num_decoder_tokens
hidden_size = 256  # Needs to be the same for both RNN's
output_size = num_decoder_tokens
dec_dropout, enc_dropout = 0.1, 0.1
num_dec_layers,num_enc_layers = 2, 2
encoder_embedding_size, decoder_embedding_size = 256, 256

In [31]:
def train(num_encoder_tokens,input_embedding_size, dp, cell_, hidden_size, num_enc_layers, num_dec_layers,num_epochs, output_size, input_size_decoder,batch_size, beam_width):
    
      #Decoder Selection
      if(cell_=="LSTM"):
          decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
            #Encoder Selection
      if(cell_=="LSTM"):
          encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp).to(device)
     
      #Model Selection
      if(cell_=="LSTM"):    
          model = Seq2Seq(encoder_net, decoder_net).to(device)

      split_dim, bs,  m_norm = 1, batch_size, 1
      optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      criterion = nn.CrossEntropyLoss()
      train_ds_y, train_ds_x = torch.split(target_data, bs, dim = split_dim), torch.split(input_data, bs, dim = split_dim)
      correct_prediction  = 0
      #print(train_ds_x)
      for epoch in range(num_epochs):
          print(f"[Epoch {epoch} / {num_epochs}]")
          model.eval()
          total_count = len(val_X)
          model.train()
          for i, (x,y) in enumerate(zip(train_ds_x,train_ds_y)):
              # Get input and targets and get to cuda
              target, inp_data = y.to(device), x.to(device)
              # Forward prop
              output = model(inp_data, target)
              target, output = target[1:].reshape(-1),  output[1:].reshape(-1, output.shape[2])
              optimizer.zero_grad()
              loss = criterion(output, target)
              loss.backward() # Back prop
              # Clip to avoid exploding gradient issues, makes sure grads are
              # within a healthy range
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = m_norm)
              # Gradient descent step
              optimizer.step()
          correct_pred, total_, lstm_bw = 0, total_count, 1
          model.eval()
          for i in range(total_):
              if(cell_=="LSTM"):
                  decoded_sentence = beam_search(model,val_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index, 
                                                 max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens,1,device)
              if decoded_sentence == val_Y[i][1:-1]:
                  correct_pred = correct_pred + 1
          test_accuracy = correct_pred / total_
          print(correct_pred / total_)  # test_accuracy = correct_pred / total_w
      
      correct_pred, total_words = 0,  len(test_X)
      dec, cor = [], []
      for i in range(total_words):
          decoded_sentence = beam_search(model,test_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index,
                              max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens,1,device)
          if decoded_sentence == test_Y[i][1:-1]:
                correct_pred = correct_pred + 1
      test_accuracy = correct_pred / total_words
      ex, inp =  [], []
      np.random.seed(10)
      print("Test_Accuracy is : ")
      print(correct_pred / total_words)
      for index in range(total_words):
          decoded_sentence = beam_search(model,test_X[index], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index,
                               max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens,1,device)

          if test_Y[index][1:-1] != decoded_sentence:
              ex.append("NO")
          else:
              ex.append("YES")        
          
          dec_out, inp_word = decoded_sentence, test_X[index]
          inp.append(inp_word)
          true_out = test_Y[index][1:-1]
          dec.append(dec_out)
          cor.append(true_out)

      grid = {
              'Input_Word': inp, 
              'Decoded_Output' : dec, 
              'True_Output' : cor,
              "Match Result" : ex
             }
      csv_path = '/content/drive/MyDrive/aksharantar_dataset/tel/Van_Pred.csv'
      df = pd.DataFrame(grid)
      df.to_csv(csv_path, index=False,header=True)
      pd.DataFrame(grid)
      print(df.head(20))

In [32]:
train(input_size_encoder ,256, 0.3, "LSTM",512, 3, 3, 30, num_decoder_tokens, num_decoder_tokens, 512, 3)

[Epoch 0 / 30]
0.0
[Epoch 1 / 30]
0.0
[Epoch 2 / 30]
0.0
[Epoch 3 / 30]
0.010986328125
[Epoch 4 / 30]
0.103271484375
[Epoch 5 / 30]
0.24951171875
[Epoch 6 / 30]
0.351806640625
[Epoch 7 / 30]
0.39697265625
[Epoch 8 / 30]
0.421630859375
[Epoch 9 / 30]
0.46044921875
[Epoch 10 / 30]
0.487060546875
[Epoch 11 / 30]
0.49853515625
[Epoch 12 / 30]
0.4931640625
[Epoch 13 / 30]
0.5009765625
[Epoch 14 / 30]
0.514892578125
[Epoch 15 / 30]
0.499755859375
[Epoch 16 / 30]
0.507080078125
[Epoch 17 / 30]
0.5419921875
[Epoch 18 / 30]
0.52001953125
[Epoch 19 / 30]
0.521728515625
[Epoch 20 / 30]
0.52685546875
[Epoch 21 / 30]
0.556640625
[Epoch 22 / 30]
0.560302734375
[Epoch 23 / 30]
0.555419921875
[Epoch 24 / 30]
0.5625
[Epoch 25 / 30]
0.561767578125
[Epoch 26 / 30]
0.55419921875
[Epoch 27 / 30]
0.5439453125
[Epoch 28 / 30]
0.546630859375
[Epoch 29 / 30]
0.56396484375
Test_Accuracy is : 
0.508544921875
           Input_Word Decoded_Output   True_Output Match Result
0          vithananni       వితనన్ని   వి